In [1]:
import os
from datetime import datetime
from typing import Dict, Tuple, Any
from tqdm import tqdm
import pickle

import math
import numpy as np
import pandas as pd

import cv2
import albumentations
from torch.utils.data import Dataset

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import OneCycleLR, ReduceLROnPlateau

import timm

In [2]:
# ---------------------------------------
# parameters

MODEL_DIR = './model_checkpoints/'
DATA_DIR = '../input/'
LOG_DIR = './logs/'
DEVICE = 'cuda:0'
MODEL_NAME = 'rexnet_200'

TRAIN_STEP = 0
FOLD = 0

IMAGE_SIZE = 256
BATCH_SIZE = 64
NUM_EPOCHS = 10
NUM_WORKERS = 4
LR = 1e-4
USE_AMP = True


In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold

In [12]:
# dfstd = pd.read_csv(os.path.join(DATA_DIR, 'train_0.csv'))
dfstd.loc[dfstd['fold'] != 0].shape

(1264376, 3)

In [4]:
dffull = pd.read_csv(os.path.join(DATA_DIR, 'train_full.csv'))

In [8]:
dffull.shape

(4132914, 3)

In [5]:
dffull['landmark_id'].nunique()

203094

In [13]:
lm, cnt = np.unique(dffull['landmark_id'], return_counts=True)

In [19]:
skf = StratifiedKFold(50, shuffle=True, random_state=233)

In [20]:
dffull['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(dffull, dffull['landmark_id'])):
    dffull.loc[valid_idx, 'fold'] = i
    break

/home/yuan/miniconda3/envs/FULL_ML/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=50.
  warnings.warn(("The least populated class in y has only %d"


In [23]:
dffull.drop(columns=['url'], inplace=True)

In [24]:
dffull.head()

,id,landmark_id,fold
0,6e158a47eb2ca3f6,142820,-1
1,202cd79556f30760,104169,-1
2,3ad87684c99c06e1,37914,-1
3,e7f70e9c61e66af3,102140,-1
4,4072182eddd0100e,2474,-1


In [27]:
dffull.loc[dffull['fold'] == 0]

,id,landmark_id,fold
24,f1556e1eeeba213f,36639,0
55,c183025976644be3,175201,0
78,33aab955d639297d,188405,0
143,5bf3cc38f1983c07,110444,0
157,f29fbc306218b459,38921,0
...,...,...,...
4132662,9bc9895227a67d07,67895,0
4132784,181eb4579fd365ae,178422,0
4132791,a01fccca4566b5db,54526,0
4132830,e33de801236df2f9,194957,0


In [34]:
vld_df = dffull.loc[dffull['fold'] == 0, 'landmark_id']

In [37]:
lm, cnt = np.unique(vld_df, return_counts=True)

In [43]:
cnt.max()

205

In [45]:
dffull.to_csv(os.path.join(DATA_DIR, 'train_full_fold.csv'))